In [ ]:
%%capture
!pip install peft datasets wandb

In [ ]:
import torch
import wandb
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import numpy as np
from sklearn.metrics import precision_score, f1_score, recall_score
from sklearn.model_selection import train_test_split

In [ ]:
# Load dataset from DataFrame
df = pd.read_csv("/content/15_labels_data.csv")  # Replace with actual file path
df = df.rename(columns={"text": "text", "label": "label"})

# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize dataset
tokens = tokenizer(list(df["text"]), padding="max_length", truncation=True, return_tensors="pt")
input_ids, attention_masks = tokens["input_ids"], tokens["attention_mask"]
labels = torch.tensor(df["label"].values)

# Split dataset
train_inp, val_inp, train_label, val_label, train_mask, val_mask = train_test_split(
    input_ids, labels, attention_masks, test_size=0.1, random_state=42
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Convert to PyTorch Dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }

train_dataset = CustomDataset(train_inp, train_mask, train_label)
eval_dataset = CustomDataset(val_inp, val_mask, val_label)
log_dir = 'dbert_model'

In [ ]:
# Ensure GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU for training")
else:
    device = torch.device("cpu")
    print("Using CPU for training")


# Initialize W&B
wandb.init(project="Spring2025")

# Define compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=15)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="/kaggle/working/logs",
    logging_steps=10,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    report_to=["wandb"],  # Enable W&B logging
    fp16=torch.cuda.is_available(),
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

Using GPU for training


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.852300,0.952618,0.700495,0.712589,0.675044
2,0.514700,0.751156,0.766223,0.775534,0.766783
3,0.257500,0.819159,0.775102,0.771971,0.770570


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=2841, training_loss=0.8125801198407404, metrics={'train_runtime': 945.9301, 'train_samples_per_second': 24.024, 'train_steps_per_second': 3.003, 'total_flos': 5979896635622400.0, 'train_loss': 0.8125801198407404, 'epoch': 3.0})

In [ ]:
# Create dummy predictions and labels
class DummyPred:
    predictions = np.array([[0.1, 0.9], [0.8, 0.2]])  # Example logits
    label_ids = np.array([1, 0])  # Example true labels

compute_metrics(DummyPred())  # Run and check output


In [ ]:
results = trainer.evaluate()
print(results)